# FlashRank reranker

>[FlashRank](https://github.com/PrithivirajDamodaran/FlashRank)는 기존 검색 및 `retrieval` 파이프라인에 재순위를 추가하기 위한 초경량 및 초고속 Python 라이브러리입니다. 이는 SoTA `cross-encoders`를 기반으로 합니다.

이 노트북은 문서 압축 및 `retrieval`을 위해 [flashrank](https://github.com/PrithivirajDamodaran/FlashRank)를 사용하는 방법을 보여줍니다.


## 환경설정

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# 설치
# !pip install -qU flashrank

In [ ]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )

## FlashrankRerank

간단한 예시를 위한 데이터를 로드하고 retriever 를 생성합니다.

In [ ]:
from langchain.schema import Document

def get_sample_docs():
    return [
        Document(
            page_content="미국의 수도는 워싱턴이고, 일본은 도쿄이며 북한은 평양이다."
        ),
        Document(
            page_content="대한민국은 동아시아에 위치한 나라로, 수도는 부산이라고 잘못 알려진 경우도 있습니다."
        ),
        Document(
            page_content="서울은 대한민국의 수도로, 정치, 경제, 문화의 중심지입니다."
        ),
        Document(
            page_content="많은 사람들이 대구를 대한민국의 수도로 착각하지만, 실제 수도는 아닙니다."
        ),
        Document(page_content="한국의 수도는 서울이며, 세계적으로 유명한 도시입니다."),
        Document(page_content="대한민국의 가장 큰 도시는 인천이지만, 수도는 아닙니다."),
        Document(
            page_content="서울은 대한민국의 수도로서, 1948년부터 공식적으로 지정되었습니다."
        ),
        Document(
            page_content="한국의 수도는 평양이라는 오해가 있을 수 있지만, 이는 북한의 수도입니다."
        ),
        Document(
            page_content="미국의 수도는 워싱턴이고, 재팬은 도교이며 코리아는 서울이다."
        ),
        Document(page_content="대한민국은 동아시에 위치한 나라이며, 분단국가이다."),
    ]

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings


vector_store = Chroma.from_documents(get_sample_docs(), OpenAIEmbeddings())
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

In [ ]:
query = "대한민국의 수도는?"

docs = retriever.invoke(query)

pretty_print_docs(docs)

이제 기본 `retriever`를 `ContextualCompressionRetriever`로 감싸고, `FlashrankRerank`를 압축기로 사용해 봅시다.

In [10]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_openai import ChatOpenAI

# LLM 초기화
llm = ChatOpenAI(temperature=0)

# 문서 압축기 초기화
compressor = FlashrankRerank(model="ms-marco-MultiBERT-L-12")

# 문맥 압축 검색기 초기화
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

# 압축된 문서 검색
compressed_docs = compression_retriever.invoke("대한민국의 수도는?")

# 문서 ID 출력
pretty_print_docs(compressed_docs)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Document 1:

미국의 수도는 워싱턴이고, 재팬은 도교이며 코리아는 서울이다.
Metadata: {'id': 6, 'relevance_score': 0.9997258}
----------------------------------------------------------------------------------------------------
Document 2:

미국의 수도는 워싱턴이고, 일본은 도쿄이며 북한은 평양이다.
Metadata: {'id': 5, 'relevance_score': 0.9996192}
----------------------------------------------------------------------------------------------------
Document 3:

한국의 수도는 평양이라는 오해가 있을 수 있지만, 이는 북한의 수도입니다.
Metadata: {'id': 0, 'relevance_score': 0.9995623}


reranker 가 적용된 후 결과를 비교합니다.